In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [4]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [5]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [6]:
scenarios = ["1. Successful Search Result Display",
"2. No Results Found",
"3. Pagination Navigation",
"4. Search Keyword Matches Shop Name",
"5. Search Keyword Matches Brand Name Without Available Shop",
"6. Search Keyword Does Not Match Any Shop or Brand",
"7. User Filters Search Results by Most Related",
"8. User Filters Search Results by Newest Products",
"9. User Filters Search Results by Best Sellers",
"10. User Filters Search Results by Price from Lowest to Highest",
"11. User Filters Search Results by Price from Highest to Lowest"]

In [7]:
prompt_uc = """
Search
Description
This feature allows users to enter search queries and view relevant product listings.
Actor: User
Preconditions: None
Postconditions: 
Basic Flow:
Step 1: User types keyword in the search bar.
Step 2: User press enter or Click search icon.
Step 3: System show list of relevant search match. Relevant search matchs are products with name or description including or related to the keywords. 
- Maximum 60 product cards are shown for each page.
- The pagination bar is at the bottom of the page, user can click on the desired page or left or right button to change to a new page.

Alterative flow: 
1. If the keyword related with a shop name; System display:
- the shop's information at the top of the search result including: shop name, number of items, ratings, and response rate, average response time
-shop product's card.
-related products card (product has name match search keyword).
2. If the keyword related with a brand name (without shop available); System display:
- the most related/best sold shop's that sell that brand at the top of the search result including: shop name, number of items, ratings, and response rate, average response time
-related products card (product has name match search keyword).
3. If the keyword not match any shop or brand; System display:
-related products card (product has name match search keyword).
4. User can filter the search result with 4 mode under the form of radio buttons: 
-result most related with the keyword (Default): Prior the exactly match by name, then partial match by name and then the match by description.
-newest product result.
-best seller result.
-sort the product by price (dropdown): from lowest to highest and from highest to lowest.

Exception flow:
If No Search Results Found, system display: random product cards.
"""

In [8]:
scenarios = ["1. Successful Search Result Display"]

In [9]:
for scenario in scenarios:
    i = 1
    while(i<=1):
        try: 
            promptTestCase = [
            { "role": "system", "content": SYSTEM_PROMPT_8},
            { "role": "user", "content": scenario + "\n" + prompt_uc}
            ]
            gpt_response = ask(promptTestCase, client, model)
            print(gpt_response)
            json_data = json.loads(gpt_response)
            write_json_to_txt_raw(json_data, scenario + "-Raw" + str(i) + ".txt")

            promptSelfRef = [
                { "role": "system", "content": SELF_REF},
                { "role": "user", "content": scenario + "\n" + gpt_response}
            ]
            final_response = ask(promptSelfRef, client, model)
            print(final_response)
            json_fin = json.loads(final_response)
            write_json_to_txt(json_fin, scenario + str(i) + ".txt")
            i+=1
        except Exception as e:
            print(f"An error occurred with scenario '{scenario}' iteration {i}: {e}")

{
  "Test Case 1": {
    "testCaseName": "Successful Search Result Display with Keyword Matching Product Name or Description",
    "objective": "Verify that the system displays a list of relevant product cards when a user searches using a keyword that matches product names or descriptions.",
    "testSteps": [
      "Step 1: Navigate to the search bar on the homepage.",
      "Step 2: Type a keyword known to match several product names or descriptions in the search bar.",
      "Step 3: Press enter or click the search icon.",
      "Step 4: Observe the search results displayed."
    ],
    "expectedResult": "A list of product cards relevant to the search keyword is displayed, with a maximum of 60 product cards per page and pagination options at the bottom.",
    "explanation": "This test case ensures that the basic flow of the search functionality works as expected, displaying relevant search results based on product names or descriptions."
  },
  "Test Case 2": {
    "testCaseName": "